In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 39 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595721 sha256=fb940b699c28d0a7f86dcf2274dddde7fbc191559c52cdbf115afc25312645a9
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.2 MB/s 
     |████████████████████████████████| 754 kB 4.3 MB/s 
     |████████████████████████████████| 3.0 MB 33.6 MB/s 
     |████████████████████████████████| 895 kB 51.9 MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-qz0ex5sk
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-qz0ex5sk
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=15f578b06974a4ed9e8e59a046add7037d52b84315d59c087a845ceed0299c0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-ii09ccub/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
dataset_train = nlp.data.TSVDataset("/content/drive/My Drive/KoBERT/data/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/drive/My Drive/KoBERT/data/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 50
batch_size = 8
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200 
learning_rate =  2e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2) #This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5,
                 dr_rate = None,
                 validation_split = 0.1,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/5164 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.6405898332595825 train acc 0.0
epoch 1 batch id 201 loss 1.5612671375274658 train acc 0.20708955223880596
epoch 1 batch id 401 loss 1.6258141994476318 train acc 0.24283042394014961
epoch 1 batch id 601 loss 1.9992477893829346 train acc 0.2576955074875208
epoch 1 batch id 801 loss 1.2825086116790771 train acc 0.27137952559300876
epoch 1 batch id 1001 loss 1.4572073221206665 train acc 0.286963036963037
epoch 1 batch id 1201 loss 1.2909042835235596 train acc 0.2981890091590341
epoch 1 batch id 1401 loss 1.4052515029907227 train acc 0.3173625981441827
epoch 1 batch id 1601 loss 1.267577886581421 train acc 0.3392410993129294
epoch 1 batch id 1801 loss 0.8705451488494873 train acc 0.3600777345918934
epoch 1 batch id 2001 loss 1.3695752620697021 train acc 0.37831084457771114
epoch 1 batch id 2201 loss 1.3963947296142578 train acc 0.39561562925942756
epoch 1 batch id 2401 loss 0.7046957612037659 train acc 0.4111307788421491
epoch 1 batch id 2601 loss 0.956589341163635

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1292 [00:00<?, ?it/s]

...

epoch 9 test acc 0.7161377708978328


  0%|          | 0/5164 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.0016458046156913042 train acc 1.0
epoch 10 batch id 201 loss 1.721928596496582 train acc 0.9720149253731343
epoch 10 batch id 401 loss 0.09381672739982605 train acc 0.9713216957605985
epoch 10 batch id 601 loss 1.1790534257888794 train acc 0.9702579034941764
epoch 10 batch id 801 loss 0.0007453353609889746 train acc 0.967852684144819
epoch 10 batch id 1001 loss 0.8331896066665649 train acc 0.9685314685314685
epoch 10 batch id 1201 loss 0.0019233082421123981 train acc 0.9685678601165695
epoch 10 batch id 1401 loss 0.0027504386380314827 train acc 0.9688615274803711
epoch 10 batch id 1601 loss 0.0009082970209419727 train acc 0.969628357276702
epoch 10 batch id 1801 loss 0.00031762165599502623 train acc 0.9693225985563576
epoch 10 batch id 2001 loss 0.0012755015632137656 train acc 0.9685157421289355
epoch 10 batch id 2201 loss 0.0010276641696691513 train acc 0.9679123125851885
epoch 10 batch id 2401 loss 0.0009892748203128576 train acc 0.9678259058725531
epoch 10

  0%|          | 0/1292 [00:00<?, ?it/s]

epoch 10 test acc 0.7154605263157895


# 모델 저장 


In [ ]:
PATH_PT = '/content/drive/My Drive/KoBERT/model/model_emo5.pt'
torch.save(model.state_dict(), PATH_PT)

In [ ]:
model_PT = BERTClassifier(bertmodel,  dr_rate=0.5) #bertmodel,  dr_rate=0.5
model_PT.load_state_dict(torch.load(PATH_PT))
model_PT.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True